In [1]:
import pandas as pd
import os
import sys
import json

In [2]:
if sys.platform.startswith("linux"):
    data_dir = "/bask/projects/v/vjgo8416-amber/data/gbif_download_standalone/"
elif sys.platform == "darwin":
    data_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/"
else:
    print("Not linux or mac!")

checklist_name = "singapore-moths-keys-nodup-small"

df = pd.read_csv(os.path.join("../species_checklists",checklist_name+".csv"))

In [13]:
df["n_imgs"] = ""
for idx, row in df.iterrows():
    
    family  = row["family_name"]
    genus   = row["genus_name"]
    species = row["gbif_species_name"]
    
    # Check if directory exists 
    species_dir = (
        os.path.join(data_dir,"gbif_images","tests",family,genus,species)
        )
    
    if os.path.isdir(species_dir):
        n_images_on_disk = len(
            [f for f in os.listdir(species_dir) if f.lower().endswith('.jpg') ]
            )
        print(f"{species} Count files method has", n_images_on_disk, "images")
        
        # Load metadata
        try:
            
            with open(os.path.join(species_dir,"meta_data.json")) as file:
            
                meta_data = json.load(file)
            
            # Count the number of images for this species
            n_imgs_metadata = 0

            for key, value in meta_data.items():
                if value.get("image_is_downloaded") == True:
                    n_imgs_metadata += 1
            print(f"{species} Count json metadata has", n_imgs_metadata, "images") 
            
            try:
                # 2nd way of counting images
                md2 = pd.read_json(
                    os.path.join(species_dir,"meta_data.json"), orient = 'index'
                    )

                md2_n_imgs_downloaded = md2["image_is_downloaded"].sum()
                print(f"{species} Count dataframe metadata has", md2_n_imgs_downloaded, "images")            

                if md2_n_imgs_downloaded != n_imgs_metadata:
                    print(f"Two ways of calculating metadata images don't match for {species_dir}")
            except Exception as e:
                print(e)
                print(f"{species} error counting dataframe way: {e}")
                        
            # Do n images match?
            if n_images_on_disk == n_imgs_metadata:
                # print(f"N images match for {species_dir}")
                pass
            else:
                print(f"N images MISmatch for {species_dir}")
        except Exception as e:
            pass
            # print(f"No metadata for {species_dir}")
            
    else:
        
        n_images_on_disk = 0
    
    # print(f"{species} has {n_images_on_disk} images")
    
    # Record this
    df.loc[idx, "n_imgs"] = n_images_on_disk
    
    

Nemophora aurisparsella Count files method has 0 images
Nemophora aurisparsella Count json metadata has 0 images
'image_is_downloaded'
Nemophora aurisparsella error counting dataframe way: 'image_is_downloaded'
Nemophora decisella Count files method has 10 images
Nemophora decisella Count json metadata has 10 images
Nemophora decisella Count dataframe metadata has 10 images
Atteva fabriciella Count files method has 10 images
Atteva fabriciella Count json metadata has 10 images
Atteva fabriciella Count dataframe metadata has 10 images
Atteva wallengreni Count files method has 10 images
Atteva wallengreni Count json metadata has 10 images
Atteva wallengreni Count dataframe metadata has 10 images
Gunda subnotata Count files method has 0 images
Gunda subnotata Count json metadata has 0 images
'image_is_downloaded'
Gunda subnotata error counting dataframe way: 'image_is_downloaded'
Ocinara albicollis Count files method has 10 images
Ocinara albicollis Count json metadata has 10 images
Ocina

In [ ]:
# Save the df
df.to_csv(os.path.join("../data_stats_files/","data_stats_"+checklist_name+".csv"))